<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Jupyter Notebooks</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

# **Query by Programme ID**

This notebook demonstrates how to use the `astroquery.eso` module to search for and download both **raw** and **reduced** HAWK-I near-infrared observations from the ESO Science Archive. We focus on data from the Galactic Center, specifically targeting **Sgr A\*** — the supermassive black hole at the heart of the Milky Way — as observed within a particular ESO observing programme.

The notebook walks through:
- Querying both the **main ESO archive table** and **instrument-specific tables**
- Inspecting metadata to understand observation conditions and data structure
- Filtering results by programme ID and other observation parameters
- Downloading **raw science frames** for custom reduction, or accessing **science-ready reduced products** for immediate analysis

This example uses observations from the highly cited **GALACTICNUCLEUS** survey:

- 🔗 [Science Portal Link (GALACTICNUCLEUS; 195.B-0283)](https://archive.eso.org/scienceportal/home?data_release_date=*:2025-05-17&dp_type=IMAGE&data_collection=195.B-0283&sort=-obs_date&s=P%2fDSS2%2fcolor&f=177.07582&fc=-1,-1&cs=J2000&av=true&ac=false&c=9,10,11,12,14,15,16,17,18,19,20,21&ta=RES&dts=true&at=201.330312,-5.255207&sr=i)
- 📄 [Data Release Description (GALACTICNUCLEUS)](https://www.eso.org/rm/api/v1/public/releaseDescriptions/175)
- 🔍  [Preview in the ESO Science Portal](https://archive.eso.org/dataset/ADP.2021-04-30T12:05:59.505) 
- 💾 [Download directly via DataPortal](https://dataportal.eso.org/dataPortal/file/ADP.2021-04-30T12:05:59.505)

This is a science-ready image centered on the Galactic Center, part of the public release, which exemplifies the exceptional resolution and depth of the GALACTICNUCLEUS survey.

This publicly released dataset provides an unprecedented view of the **central parsecs of the Milky Way**, combining high spatial resolution with deep near-infrared sensitivity. The data enable studies of stellar populations, extinction, gas dynamics, and feedback processes in one of the most extreme environments in the Galaxy.

<hr style="border:2px solid #0281c9"> </hr>

# **Importing and basic usage of astroquery.eso** 

In [1]:
import astroquery # import astroquery
print(f"astroquery version: {astroquery.__version__}") # check the version of astroquery

astroquery version: 0.4.11.dev10245


In [2]:
from astroquery.eso import Eso # import the ESO module from astroquery

In [3]:
eso = Eso() # create an instance of the ESO class 

# **Performing a Search for Program ID**

Here we now define the programme ID we are interested in, which is `195.B-0283` for the **GALACTICNUCLEUS** survey.

In [4]:
instrument = "HAWKI"       # the instrument to HAWKI
prog_id    = "195.B-0283"  # the program ID
dp_cat     = "SCIENCE"     # the data product category to SCIENCE

In [5]:
eso.maxrec = 1000    # For this example we limit the number of records to 1000

## **Reduced: General data query**

Quick check if the survey is in the archive... 

In [6]:
surveys = eso.list_surveys() # list all collections
prog_id in surveys # check if the program ID is in the list of surveys

True

Now the suvery can be simply queried via the following. 

In [7]:
table_reduced = eso.query_surveys(prog_id) # query the survey with the program ID

table_reduced["target_name", "s_ra", "s_dec", "proposal_id", "instrument_name", "dp_id"][:3] # print the first 3 rows of the table

target_name,s_ra,s_dec,proposal_id,instrument_name,dp_id
,deg,deg,,,
object,float64,float64,object,object,object
FAST-SGRA-10,266.58929,-28.788888,195.B-0283(A),HAWKI,ADP.2021-04-30T12:05:58.577
FAST-SGRA-10,266.55093,-28.844495,195.B-0283(A),HAWKI,ADP.2021-04-30T12:05:58.579
FAST-SGRA-10,266.5264,-28.831477,195.B-0283(A),HAWKI,ADP.2021-04-30T12:05:58.581


> **Note:** This approach assumes that the survey name is identical to the programme ID, which is **not always the case**. A more robust option is to perform a free-text search and explicitly constrain by the `proposal_id` column, as shown below.
>
> For example, the **KiDS** survey uses the programme ID `177.A-3016`, but its survey name is `KiDS` (not `177.A-3016`).  
> 📄 You can read more in the [KiDS survey release description](https://www.eso.org/rm/api/v1/public/releaseDescriptions/229).

In [8]:
table_reduced = eso.query_surveys(column_filters={"proposal_id":f"like '%{prog_id}%'"}) # query the survey with the program ID

table_reduced["target_name", "s_ra", "s_dec", "proposal_id", "instrument_name", "dp_id"][:3] # print the first 3 rows of the table

target_name,s_ra,s_dec,proposal_id,instrument_name,dp_id
,deg,deg,,,
object,float64,float64,object,object,object
2MASS_90860,185.361175,-0.102492,195.B-0283(A),HAWKI,ADP.2016-07-29T13:44:03.114
2MASS_90860,185.417579,-0.120564,195.B-0283(A),HAWKI,ADP.2016-07-29T13:44:03.115
2MASS_90860,185.417543,-0.120434,195.B-0283(A),HAWKI,ADP.2016-07-29T13:44:03.122


**Note:** In the above example, we filtered `prog_id` using `like '%{prog_id}%'` instead of just using `195.B-0283` only. 

As the project contains multiple runs such that the project code of these runs is as follows: `195.B-0283(A)`, `195.B-0283(B)`, `195.B-0283(C)` etc, thus we must use `like` in the `ADQL` syntax. 

More generally `column_filters` can be applied as follows: 

``` python
column_filters = {
                  'some_int_column': "< 5",
                  'some_float_column_2': ">= 1.23",
                  'some_char_column': "like '%John%'",
                  'some_generic_column': "in ('mango', 'apple', 'kiwi')",
                  'other_generic_column': "between '2024-01-01' and '2024-12-31'"
                }
``` 

## **Raw: Instrument specific data query**

In [9]:
table_raw = eso.query_instrument(instrument, 
                            column_filters={"prog_id":f"like '%{prog_id}%'", 
                                            "dp_cat":dp_cat})

table_raw["object", "ra", "dec", "prog_id", "instrument", "dp_id"][:3] # print the first 3 rows of the table

/opt/anaconda3/envs/astroquery_eso/lib/python3.13/site-packages/pyvo/dal/query.py:341: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


object,ra,dec,prog_id,instrument,dp_id
,deg,deg,,,
object,float64,float64,object,object,object
FAST-SGRA-1,266.414237,-29.00781,195.B-0283(A),HAWKI,HAWKI.2015-06-06T01:01:27.257
FAST-SGRA-1,266.419237,-29.00547,195.B-0283(A),HAWKI,HAWKI.2015-06-06T01:05:18.444
FAST-SGRA-1,266.415546,-29.00847,195.B-0283(A),HAWKI,HAWKI.2015-06-06T01:09:48.394


## **Raw: General data query**

In [10]:
table_raw = eso.query_main(instrument, 
                        column_filters={"prog_id":f"like '%{prog_id}%'", 
                                        "dp_cat":dp_cat})

table_raw["object", "ra", "dec", "prog_id", "instrument", "dp_id"][:3] # print the first 3 rows of the table

object,ra,dec,prog_id,instrument,dp_id
,deg,deg,,,
object,float64,float64,object,object,object
FAST-SGRA-29,266.35917694,-29.24629,195.B-0283(B),HAWKI,HAWKI.2016-03-27T09:23:56.772
FAST-SGRA-29,266.35194,-29.24488,195.B-0283(B),HAWKI,HAWKI.2016-03-27T09:25:25.617
FAST-SGRA-29,266.36155611,-29.25191,195.B-0283(B),HAWKI,HAWKI.2016-03-27T09:24:40.571


# **Downloading datasets from the archive**

The returned table has an ``dp_id`` column. It can be used to retrieve the datasets with ``retrieve_data()``.

The file names, returned in ``data_files``, points to the decompressed datasets (without the .Z extension) that have been locally downloaded. 

The default location (in the ``astropy`` cache) of the decompressed datasets can be adjusted by providing a destination keyword in the call to ``retrieve_data()``. By default, if a requested dataset is already found, it is not downloaded again from the archive. To force the retrieval of data that are present in the destination directory, use ``continuation=True`` in the call to ``retrieve_data()``.

In [11]:
# data_files = eso.retrieve_data(table_raw["dp_id"][0]) # download the first data product
# data_files = eso.retrieve_data(table['dp_id'][0], continuation=True) # force the download of all files even if present in the destination directory
# data_files = eso.retrieve_data(table['dp_id'][0], destination='./') # specify the destination directory for the download

<hr style="border:2px solid #0281c9"> </hr>